In [1]:
cd /home/urwa/Documents/Projects/NYU Remote/UrbanTrafficPrediction/

/home/urwa/Documents/Projects/NYU Remote/UrbanTrafficPrediction


In [2]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [3]:
from utils.lstm_utils import prepare_data_lstm, lstm_monthly_dataloader
from utils.lstm_utils import get_device

from utils.lstm_utils import evaluate_edge_monthy
from models.models import LSTM

from utils.lstm_utils import train_one_epoch, store_chekpoint, load_chekpoint
from utils.lstm_utils import evaluate_lstm_pipeline_model, run_inference

In [4]:
torch.manual_seed(2020)
np.random.seed(2020)

In [5]:
config = {'lr': 0.00034439316653688684,
 'layers': 3,
 'step_size': 11,
 'gamma': 0.761795969995615,
 'bptt': 19,
 'dropout': 0.1227497445640586}

In [6]:
device = get_device(cuda=True)
device

device(type='cuda')

In [7]:
data_path = '/home/urwa/Documents/Projects/NYU Remote/data/featureData/com_jfk.csv'
weights_path = '/home/urwa/Documents/Projects/NYU Remote/data/featureData/com_jfk_weights.csv'
test_data_path='/home/urwa/Documents/Projects/NYU Remote/data/featureData/jfk.csv'

exp_dir = '/home/urwa/Documents/Projects/NYU Remote/UrbanTrafficPrediction/data/lstm_12fold/jfk'

In [8]:
dataset, targetColumns, features_cols = prepare_data_lstm(data_path)

Raw Shape:  (8757, 113)
Cleaned Shape:  (8757, 38)
Target columns: 24
Feature coumns:  13


In [10]:
bptt = config['bptt']

R2List = []
EdgeR2List = []
residual_list = []


for m in range(9,13):
    
    print('-------------------------------------------------')
    print('-------------------------------------------------')
    print("Month: ", m)

    train_inout_seq, test_inout_seq = lstm_monthly_dataloader(dataset,features_cols, targetColumns, m,
                                                              bptt, device)
    
    lstm_layers = config['layers']
    network_size = len(targetColumns)
    feat_size = len(features_cols)
    dropout = config['dropout']
    hidden_layer_size=100
    
    
    model = LSTM(feat_size, network_size, hidden_layer_size, lstm_layers, dropout).to(device)
    loss_function = nn.L1Loss()   
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma']) 
    
    
    
    epochs = 80
    if m == 8:
        start_epoch=37
        best_r2=0.813
        model,optimizer,scheduler = load_chekpoint(exp_dir, model,optimizer,scheduler)
        
    else:
        start_epoch=0
        best_r2=0
        
    
    print("\n Training model...")
    for i in range(start_epoch,epochs):
        
        loss = train_one_epoch(model, optimizer, loss_function, train_inout_seq, device)
        scheduler.step()
        store_chekpoint(exp_dir, model, optimizer, scheduler)

        residual, r2, rmse, mae = evaluate_lstm_pipeline_model(model, test_inout_seq, device)
        print(f'epoch: {i:3} loss: {loss:10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

        if r2 > best_r2:
            best_r2 = r2
            best_residual = residual
            torch.save(model.state_dict(), os.path.join(exp_dir, str(m)+'.pt'))
            np.save(os.path.join(exp_dir,str(m)+'.npy'), best_residual)
            edge_res, edge_r2, edge_rmse, edge_mae = evaluate_edge_monthy(model, test_inout_seq, device, 
                                                                   targetColumns, weights_path, 
                                                              test_data_path, m, bptt)
            print('Saving model and residual')
            print('edge r2: ', edge_r2)
            
            
    print("best_r2: ", best_r2)

    R2List.append(best_r2)
    EdgeR2List.append(edge_r2)
    residual_list.append(best_residual)

-------------------------------------------------
-------------------------------------------------
Month:  9
train test split
train shape:  (8037, 38)
test shape:  (720, 38)
train feature tensor shape : torch.Size([8037, 13])
train target tensor shape : torch.Size([8037, 24])
test feature tensor shape : torch.Size([720, 13])
test target tensor shape : torch.Size([720, 24])

sequences
torch.Size([19, 13]) torch.Size([19, 24]) torch.Size([19, 24])

 Training model...
------- Saving checkpoint---------------
epoch:   0 loss: 9.12563621 r2: 0.710 rmse: 188.630 mae: 6.279
Saving model and residual
edge r2:  0.48174747341632457
------- Saving checkpoint---------------
epoch:   1 loss: 5.63439954 r2: 0.827 rmse: 112.703 mae: 5.305
Saving model and residual
edge r2:  0.5611974697031603
------- Saving checkpoint---------------
epoch:   2 loss: 5.20744470 r2: 0.838 rmse: 105.123 mae: 5.247
Saving model and residual
edge r2:  0.5676285396098836
------- Saving checkpoint---------------
epoch:   3

epoch:  71 loss: 3.35275538 r2: 0.853 rmse: 95.236 mae: 5.123
------- Saving checkpoint---------------
epoch:  72 loss: 3.34850436 r2: 0.850 rmse: 97.194 mae: 5.151
------- Saving checkpoint---------------
epoch:  73 loss: 3.34777514 r2: 0.853 rmse: 95.370 mae: 5.121
------- Saving checkpoint---------------
epoch:  74 loss: 3.34494877 r2: 0.853 rmse: 95.852 mae: 5.121
------- Saving checkpoint---------------
epoch:  75 loss: 3.34114791 r2: 0.854 rmse: 95.154 mae: 5.112
------- Saving checkpoint---------------
epoch:  76 loss: 3.34034756 r2: 0.853 rmse: 95.344 mae: 5.122
------- Saving checkpoint---------------
epoch:  77 loss: 3.33764008 r2: 0.855 rmse: 94.478 mae: 5.100
------- Saving checkpoint---------------
epoch:  78 loss: 3.33147644 r2: 0.853 rmse: 95.440 mae: 5.115
------- Saving checkpoint---------------
epoch:  79 loss: 3.32846876 r2: 0.854 rmse: 94.769 mae: 5.107
best_r2:  0.8643532084605895
-------------------------------------------------
-----------------------------------

------- Saving checkpoint---------------
epoch:  62 loss: 3.39707400 r2: 0.872 rmse: 89.827 mae: 4.929
------- Saving checkpoint---------------
epoch:  63 loss: 3.39307833 r2: 0.868 rmse: 92.978 mae: 4.987
------- Saving checkpoint---------------
epoch:  64 loss: 3.38994821 r2: 0.868 rmse: 92.406 mae: 4.981
------- Saving checkpoint---------------
epoch:  65 loss: 3.38738992 r2: 0.869 rmse: 91.905 mae: 4.970
------- Saving checkpoint---------------
epoch:  66 loss: 3.38343179 r2: 0.869 rmse: 92.326 mae: 4.973
------- Saving checkpoint---------------
epoch:  67 loss: 3.37308661 r2: 0.867 rmse: 93.041 mae: 4.985
------- Saving checkpoint---------------
epoch:  68 loss: 3.36879019 r2: 0.867 rmse: 93.174 mae: 4.990
------- Saving checkpoint---------------
epoch:  69 loss: 3.36813014 r2: 0.869 rmse: 92.149 mae: 4.969
------- Saving checkpoint---------------
epoch:  70 loss: 3.36448962 r2: 0.867 rmse: 93.087 mae: 4.980
------- Saving checkpoint---------------
epoch:  71 loss: 3.36323120 r2: 

------- Saving checkpoint---------------
epoch:  53 loss: 3.43686889 r2: 0.793 rmse: 141.500 mae: 5.715
------- Saving checkpoint---------------
epoch:  54 loss: 3.43304188 r2: 0.797 rmse: 138.374 mae: 5.662
------- Saving checkpoint---------------
epoch:  55 loss: 3.42635249 r2: 0.797 rmse: 138.471 mae: 5.671
------- Saving checkpoint---------------
epoch:  56 loss: 3.41498116 r2: 0.799 rmse: 137.524 mae: 5.667
------- Saving checkpoint---------------
epoch:  57 loss: 3.40904716 r2: 0.795 rmse: 139.792 mae: 5.687
------- Saving checkpoint---------------
epoch:  58 loss: 3.40214953 r2: 0.794 rmse: 140.677 mae: 5.685
------- Saving checkpoint---------------
epoch:  59 loss: 3.40258859 r2: 0.794 rmse: 140.600 mae: 5.695
------- Saving checkpoint---------------
epoch:  60 loss: 3.39818042 r2: 0.794 rmse: 141.044 mae: 5.707
------- Saving checkpoint---------------
epoch:  61 loss: 3.39340975 r2: 0.792 rmse: 142.250 mae: 5.720
------- Saving checkpoint---------------
epoch:  62 loss: 3.3910

------- Saving checkpoint---------------
epoch:  43 loss: 3.53998447 r2: 0.801 rmse: 106.440 mae: 5.212
------- Saving checkpoint---------------
epoch:  44 loss: 3.53569263 r2: 0.797 rmse: 108.826 mae: 5.241
------- Saving checkpoint---------------
epoch:  45 loss: 3.51203191 r2: 0.806 rmse: 104.133 mae: 5.181
------- Saving checkpoint---------------
epoch:  46 loss: 3.50727785 r2: 0.797 rmse: 108.804 mae: 5.276
------- Saving checkpoint---------------
epoch:  47 loss: 3.49945484 r2: 0.797 rmse: 108.511 mae: 5.249
------- Saving checkpoint---------------
epoch:  48 loss: 3.49605662 r2: 0.801 rmse: 106.373 mae: 5.233
------- Saving checkpoint---------------
epoch:  49 loss: 3.48996543 r2: 0.800 rmse: 106.872 mae: 5.239
------- Saving checkpoint---------------
epoch:  50 loss: 3.48520399 r2: 0.801 rmse: 106.532 mae: 5.223
------- Saving checkpoint---------------
epoch:  51 loss: 3.48247461 r2: 0.801 rmse: 106.513 mae: 5.243
------- Saving checkpoint---------------
epoch:  52 loss: 3.4789